# Mapping Journey: ICD-11 to CIEL Concepts

## Introduction

Completing the mappings of diagnoses and clinical findings to SNOMED CT and ICD-10 was relatively easier. Andrew was able to leverage some professional connections to obtain curated data, which significantly improved the accuracy of mappings between CIEL concepts and these terminologies. In return, we committed to delivering mappings to ICD-11.

Since ICD-11 is relatively recent — not necessarily in its creation, but in its implementation — and is much more complex than ICD-10 due to its use of extension codes, pre-coordinated, and post-coordinated concepts, the mapping posed additional challenges. It is important to note that CIEL is based exclusively on pre-coordinated concepts. Thus, we began the ambitious task of mapping the remaining **27,796 CIEL diagnosis concepts** to ICD-11.

Previously, manual mapping efforts were extremely time-consuming. Before the initiation of this project, it could take up to **5 minutes** to manually map 3 CIEL concepts using a labor-intensive approach of copying, pasting, and searching for references across multiple tabs. Based on this method, a skilled individual could theoretically map the remaining concepts in approximately **193 days** (considering 1.67 minutes per mapping and 4 working hours per day).

## Phase 1: Leveraging Existing Resources and Developing Support Tools

At this time, there was a parallel effort by the OCL [(Open Concept Lab)](https://app.openconceptlab.org/) community to develop a mapping support tool, known as the **Mapping Tool**. This tool aimed to assist the mapping process across various terminologies, combining the power of large language models (LLMs) and ElasticSearch to suggest likely target concepts for a given source concept. Although the Mapping Tool was not a production-grade tool, we were able to take advantage of its development to create an initial triage table with approximately **1,693 ICD-11 code suggestions**.

Since the Mapping Tool provided only **code suggestions** — and not the **mapping relationship type** (such as *NARROWER-THAN* or *BROADER-THAN*) — it operated under the assumption of *SAME-AS*, which was often not accurate. Therefore, it was necessary to curate these data points further.

We imported these suggestions into a web service that was already deployed to support CIEL improvement processes, known as **CIEL Lab**.

For the ICD-11 mapping, the **CIEL Lab** became a large working table where each row represented a CIEL concept. Concepts were either sourced from the Mapping Tool or added manually. The table allowed users to correct a suggested code, provide a missing code, and define the mapping type as *SAME-AS*, *NARROWER-THAN*, or *BROADER-THAN*.

| Concept ID | FSN | ICD-10 Code | ICD-10 Name | ICD-11 Code | ICD-11 Name | Map Type | Actions |
|------------|-----|-------------|-------------|-------------|-------------|----------|------|
| Integer ID (clickable link to OCL) | Full Specified Name of CIEL Concept | Mapped WHO-ICD-10 code | ICD-10 FSN captured from secondary lookup table | Input field for suggested ICD-11 code (auto-filled if available) | Captured via ICD-11 API | Radio buttons to select mapping type: SAME-AS, NARROWER-THAN, BROADER-THAN | Final action buttons:<br>- **Send to Review** (stores in review table for later update to CIEL database)<br>- **Send to Manual Queue** (for concepts requiring two ICD-11 codes; removed from pending list and added to manual queue) |

Each ICD-11 code field is supported by three auxiliary buttons:

- **Run Linearization Search** (using the ICD API) using `/icd/release/11/{releaseId}/{linearizationname}/search` endpoint
- **Run Autocode** (using the ICD API) using `/icd/entity/autocode` endpoint
- **Run Cross Reference Mapping** using the WHO ICD-10 to ICD-11 cross-reference mapping

Each button opens a modal with analysis options, and the best match found in the search can be automatically populated into the input field.


### 1.1 Developing the Environment for CIEL Lab Support Tools

### 1.1.1 Installing the ICD API Using Docker

To support the **Run Linearization Search** and **Run Autocode** functionalities, it was necessary to install the **ICD-11 API** locally using Docker. The setup followed the official WHO instructions available at [ICD API Docker Container Documentation](https://icd.who.int/docs/icd-api/ICDAPI-DockerContainer/).

```sh
docker run -d \
  --name icd-api \
  -p 8887:80 \
  -e acceptLicense=true \
  -e saveAnalytics=true \
  whoicd/icd-api
```

In our setup, an **Nginx proxy** was configured to distribute the API under a DNS name. However, for reproducibility and ease of use, we also propose the creation of a `vscode dev container` file that can deploy everything required to run the sandbox environment locally.

You just need to enter in your vscode, press `Cmd + Shift + P`, search and select `Open Workspace in Container`

With the local ICD API available, our tools were able to automatically interact with ICD-11 services for term searches and auto-coding suggestions without the need of authentication like the WHO API and without speed and requests limitations.

### 1.1.2 Creating Supporting Tables for ICD-10 to ICD-11 Cross Reference

To enable the **Run Cross Reference Mapping** functionality, we needed access to cross-reference data between ICD-10 and ICD-11.

We downloaded the mapping archive from the [official WHO site](https://icd.who.int/browse/2025-01/mms/en) under:
- `Info > ICD-10 / ICD-11 Mapping Tables`,  
or directly from this link: [Download Mapping Tables (ZIP)](https://icdcdn.who.int/static/releasefiles/2025-01/mapping.zip).

After extracting the files, we executed the following shell script to import the mapping data into a MySQL database:

```sh
cd icd11_mapping_tool
wget https://icdcdn.who.int/static/releasefiles/2025-01/mapping.zip
unzip mapping.zip -d mapping
sudo chmod +x seed_icd_cross_reference_tables.sh
./seed_icd_cross_reference_tables.sh -u root -proot -d sandbox -h db-sandbox
```

#### Key Table for Cross-Reference Mapping

The most relevant table for searches is `icd11_10To11MapToMultipleCategories` as it contains the mappings of ICD-10 codes to multiple corresponding ICD-11 codes.

#### Additional View for ICD-10 Codes

In order to assist with future joins, especially when needing the ICD-10 FSN (Full Specified Name), we created a simple view: